<a href="https://colab.research.google.com/github/tomonari-masada/courses/blob/master/10_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習で文書分類を試みる

* WikipediaからUSの男性俳優と女性俳優のページをクローリングして簡単な前処理を済ませてあるデータを使う。

* 分析の目的1: 検証データでできるかぎりチューニングを行い、最後にテストデータでの分類性能を明らかにする。

* 分析の目的2： 男性俳優と女性俳優のページを分類する際に、どのような単語が特に効いているかを調べる。

 * この調査によって、俳優に関する記述におけるジェンダー・ステレオタイプを明らかにできるか？

In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

### 1) データファイルを読み込む

* データファイルは、あらかじめ自分のGoogle Driveの適当なフォルダに置いておく。

* データファイルの各行には、女性俳優(1)か男性俳優(0)かを表すフラグ、俳優の名前、Wikipediaのページの本文が、この順に格納されている。

* データファイルの各行に対してeval組み込み関数を適用すると、Pythonのリストに変換できるようなフォーマットで、ファイルに記録されている。

In [2]:
y = list()
names = list()
corpus = list()
with open('drive/My Drive/data/us_actors_and_actresses.txt', 'r') as f:
    for line in f:
        flag, name, text = eval(line.strip())
        y.append(int(flag))
        names.append(name)
        corpus.append(text)

In [3]:
# corpusは、本文の文字列がたくさん入っているリスト。
corpus[:10]

['De Aundre Bonds born March 19 1976 is an American actor Bonds has mostly appeared as a guest actor on television shows however he was also featured in the Spike Lee film Get on the Bus Tales from the Hood and the Rick Famuyiwa film The Wood He has a production company called Take Off Productions with fellow actor Francis Capra 1 Bonds was born in Los Angeles California In 2001 he was convicted of manslaughter in the death of his aunt s boyfriend and in 2011 he finished serving a 10 year sentence at California Rehabilitation Center Norco California 2',
 'James McLaughlin also James W MacLaughlin an J W McLaughlin was an American film actor and director',
 'Dudley Dickerson November 27 1906 September 23 1968 was an American film actor Born in Chickasha Oklahoma he appeared in nearly 160 films between 1932 and 1952 and is best remembered for his roles in several Three Stooges films Given the era in which Dickerson performed he was usually cast in stereotypical roles that were common in 

In [4]:
# yは男性俳優か女性俳優かを表す0/1のresponse。
# namesは俳優の名前。
y = np.array(y)
names = np.array(names)
print(names[:20])

['De%27Aundre_Bonds' 'James_McLaughlin_(actor)' 'Dudley_Dickerson'
 'Charles_P._Thompson' 'Gus_Saville' 'Paris_Themmen' 'Peter_Haskell'
 'Robert_Keith_(actor)' 'Thomas_Carroll_(martial_artist)'
 'Matthew_Dickens' 'Nat_Benchley' 'Richard_Hunt_(puppeteer)'
 'Thomas_McDonell' 'Paul_Smith_(American_actor,_born_1929)' 'Jim_Zulevic'
 'Rob_Moran' 'Harry_Bannister' 'Shaun_Weiss' 'Peter_Sarsgaard'
 'Palmer_Williams_Jr.']


In [5]:
# Nは全文書数。
N = len(y)
print('We have {:d} documents.'.format(N))

We have 19645 documents.


In [6]:
# 訓練データとテストデータを分割するために、インデックスをランダムにシャッフルする。
indices = np.arange(N)
np.random.seed(123) # 乱数のシードを固定して再現性を持たせている。
np.random.shuffle(indices)
print(indices)

[12617 18144 17561 ... 15377 17730 15725]


In [7]:
# 手動で訓練データ8割、テストデータ2割に分割する。
border = N * 8 // 10
print('We have {:d} test documents.'.format(N - border))
train_indices = indices[:border]
test_indices = indices[border:]
y_train = y[train_indices]
y_test = y[test_indices]
names_train = names[train_indices]
names_test = names[test_indices]

We have 3929 test documents.


In [8]:
# 本文のほうも、インデックスの同じ分割を使って、訓練データとテストデータに分割する。
corpus_train = list()
for i in train_indices:
  corpus_train.append(corpus[i])
corpus_test = list()
for i in test_indices:
  corpus_test.append(corpus[i])

（ここまでのtraining setとtest setへの分割は、変えないようにしてください。）

### 2) TF-IDFで各文書をベクトル化する

* TF-IDFは単語列をベクトル化する方法のひとつ。

* ベクトルの次元は語彙数となる。各Wikipediaのページがひとつのベクトルへ変換される。

* TfidfVectorizerのパラメータをチューニングしても構わない

 * ここでTF-IDFの計算をするときにテストデータは使っていないので、ズルはしていない。

 * min_dfは、その数より少ない文書にしか出現しない単語を削除する、という意味のパラメータ。希少な単語を削除するために使う。

 * max_dfは、0から1の間の実数で指定すると、その割合より多い文書に出現する単語を削除する、という意味のパラメータ。ありふれた単語を削除するために使う。

In [9]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=100)
X_train = vectorizer.fit_transform(corpus_train)
print('# X_train shape', X_train.shape)

# X_train shape (15716, 6033)


In [10]:
# 語彙を取得する。
vocab = np.array(vectorizer.get_feature_names())

In [11]:
# 語彙の一部を見てみる（アルファベット順に並んでいるようだ）。
print(vocab[1000:1010])

['bringing' 'brings' 'britain' 'british' 'broad' 'broadcast'
 'broadcasting' 'broadcasts' 'broadway' 'broderick']


### 3) 検証データの準備

* k-fold交差検証に書き換えても良い。

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.10, random_state=42)

### 4) ロジスティック回帰の学習

* ロジスティック回帰のハイパーパラメータは、検証データでの分類性能ができるだけよくなるように、変更する。

In [13]:
# これは単純な実行例にすぎません。正則化も含めてチューニングしてください。
# TfidfVectorizerのパラメータも併せてチューニングしていいです。

clf = LogisticRegression(max_iter=1000, solver='liblinear', random_state=123)
clf.fit(X_train, y_train)
print('validation mean accuracy: {}'.format(clf.score(X_valid, y_valid)))

validation mean accuracy: 0.8829516539440203


### 5) 学習済みのロジスティック回帰モデルについて、テストデータで最終的な評価をおこなう

In [14]:
X_test = vectorizer.transform(corpus_test)
print('test mean accuracy: {}'.format(clf.score(X_test, y_test)))

test mean accuracy: 0.8788495800458132


### 6) SVMの学習

In [15]:
# これは単純な実行例にすぎません。ハイパーパラメータのチューニングもしてください。

svm = LinearSVC(C=0.3, random_state=123)
svm.fit(X_train, y_train)
print('validation mean accuracy: {}'.format(svm.score(X_valid, y_valid)))

validation mean accuracy: 0.8924936386768448


### 分類に効いている単語を調べる

* 訓練データが最も数が多いので、訓練データの分類に最も効いている単語100語を調べる。

* 下に示すのは、あくまで一つの方法にすぎない。他にどんな方法があるか調べて、その方法を実践する。

 * 下の手法の欠点は、男性俳優の文書に特徴的な単語と、女性俳優の文書に特徴的な単語とを、区別できない点である。

 * ヒント： 「svm important features」 あたりでググってみる。

In [16]:
# sklearnにあるrecursive feature eliminationという特徴量選択の手法を使ってみる。
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

from sklearn.feature_selection import RFE

rfe = RFE(estimator=clf, n_features_to_select=200, step=100)
rfe.fit(X_train, y_train)
ranking = rfe.ranking_
print(vocab[ranking == 1])

['actresses' 'adam' 'age' 'alice' 'amanda' 'amy' 'andrew' 'angela' 'anime'
 'ann' 'anna' 'anne' 'annie' 'announcer' 'anthony' 'army' 'attack' 'aunt'
 'baby' 'band' 'baseball' 'beauty' 'ben' 'billy' 'birth' 'blonde' 'bobby'
 'boy' 'boys' 'breast' 'brother' 'brothers' 'captain' 'catherine'
 'character' 'charles' 'cheerleader' 'chris' 'christopher' 'cindy'
 'claire' 'contract' 'corps' 'dance' 'dancer' 'daniel' 'daughter' 'dave'
 'david' 'debut' 'detective' 'died' 'directed' 'directing' 'director'
 'divorced' 'dorothy' 'eddie' 'edward' 'elizabeth' 'emily' 'eric' 'eve'
 'fashion' 'father' 'female' 'florence' 'football' 'frances' 'frank'
 'fraternity' 'game' 'gang' 'gay' 'george' 'gina' 'girl' 'girlfriend'
 'girls' 'grace' 'grandmother' 'guitar' 'guy' 'heather' 'helen' 'hero'
 'heroine' 'husband' 'ii' 'irene' 'jack' 'james' 'jane' 'jason' 'jay'
 'jean' 'jeff' 'jennifer' 'jenny' 'jessica' 'jim' 'jimmy' 'joe' 'john'
 'johnny' 'jonathan' 'joseph' 'joy' 'jr' 'julie' 'katie' 'kevin' 'lady'
 'larr

In [17]:
rfe = RFE(estimator=svm, n_features_to_select=200, step=100)
rfe.fit(X_train, y_train)
ranking = rfe.ranking_
print(vocab[ranking == 1])

['abby' 'actresses' 'alice' 'amanda' 'amy' 'andrew' 'angela' 'anime' 'ann'
 'anna' 'anne' 'announcer' 'anthony' 'army' 'attack' 'aunt' 'baby'
 'baseball' 'ben' 'beth' 'betsy' 'birth' 'blonde' 'bobby' 'boy' 'boys'
 'breast' 'brother' 'brothers' 'captain' 'catherine' 'charles'
 'cheerleader' 'christopher' 'cindy' 'claire' 'contract' 'corps' 'dancer'
 'daniel' 'daughter' 'dave' 'david' 'debut' 'died' 'directing' 'director'
 'disease' 'divorced' 'dorothy' 'driver' 'eddie' 'edward' 'eleanor'
 'elizabeth' 'emily' 'enlisted' 'eric' 'eve' 'father' 'female' 'florence'
 'football' 'frank' 'fraternity' 'gay' 'gertrude' 'gina' 'girl'
 'girlfriend' 'girls' 'grace' 'grandmother' 'guitar' 'guy' 'hamlet'
 'heather' 'helen' 'hero' 'heroine' 'husband' 'ii' 'irene' 'jack' 'jake'
 'james' 'jane' 'jay' 'jeff' 'jennifer' 'jenny' 'jessica' 'jim' 'joe'
 'john' 'johnny' 'jonathan' 'joseph' 'joy' 'jr' 'julie' 'justin' 'karate'
 'kathleen' 'kathy' 'katie' 'kevin' 'kimberly' 'lady' 'laura' 'lesbian'
 'lily' 'lind